In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Configuration
DATA_DIR = 'images'  # expects subfolders per class
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 15
MODEL_PATH = 'keras_model.h5'
LABELS_PATH = 'labels.txt'

if not os.path.isdir(DATA_DIR):
    raise SystemExit(f"Data directory '{DATA_DIR}' not found. Create it and add class subfolders, e.g. images/person1, images/person2")

# Count classes
class_names = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
class_names.sort()
print(f"Detected classes: {class_names}")

# Save labels
with open(LABELS_PATH, 'w', encoding='utf-8') as f:
    for name in class_names:
        f.write(name + '\n')
print(f"Saved labels to {LABELS_PATH}")

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = len(class_names)

# Simple CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    BatchNormalization(),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

checkpoint = ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1)
stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[checkpoint, stop]
)

print(f"Training complete. Best model saved to {MODEL_PATH}")


Detected classes: ['bb', 'bb1', 'bb2', 'dhanush', 'jaipreet']
Saved labels to labels.txt
Found 761 images belonging to 5 classes.
Found 190 images belonging to 5 classes.


c:\Users\bhuva\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 220, 220, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 110, 110, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 110, 110, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 108, 108, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 108, 108, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 106, 106, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 53, 53, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 53, 53, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 51, 51, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 51, 51, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 80000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    20,480,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,621,861 (78.67 MB)

...

48/48 ━━━━━━━━━━━━━━━━━━━━ 22s 465ms/step - accuracy: 0.9067 - loss: 0.4773 - val_accuracy: 0.1053 - val_loss: 50.1440
Epoch 8/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step - accuracy: 0.9114 - loss: 0.3784
Epoch 8: val_accuracy did not improve from 0.10526
48/48 ━━━━━━━━━━━━━━━━━━━━ 24s 496ms/step - accuracy: 0.9133 - loss: 0.4047 - val_accuracy: 0.0526 - val_loss: 143.0352
Epoch 9/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step - accuracy: 0.9092 - loss: 0.3462
Epoch 9: val_accuracy did not improve from 0.10526
48/48 ━━━━━━━━━━━━━━━━━━━━ 24s 504ms/step - accuracy: 0.9067 - loss: 0.4933 - val_accuracy: 0.0895 - val_loss: 58.5378
Epoch 10/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.9130 - loss: 0.5046
Epoch 10: val_accuracy improved from 0.10526 to 0.43158, saving model to keras_model.h5


48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 510ms/step - accuracy: 0.9093 - loss: 0.4272 - val_accuracy: 0.4316 - val_loss: 23.3152
Epoch 11/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step - accuracy: 0.9073 - loss: 0.3692
Epoch 11: val_accuracy improved from 0.43158 to 0.70000, saving model to keras_model.h5


48/48 ━━━━━━━━━━━━━━━━━━━━ 26s 531ms/step - accuracy: 0.9159 - loss: 0.4673 - val_accuracy: 0.7000 - val_loss: 5.2604
Epoch 12/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step - accuracy: 0.8788 - loss: 1.0218
Epoch 12: val_accuracy did not improve from 0.70000
48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 519ms/step - accuracy: 0.8922 - loss: 1.1870 - val_accuracy: 0.6000 - val_loss: 10.3028
Epoch 13/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 502ms/step - accuracy: 0.8985 - loss: 1.2775
Epoch 13: val_accuracy improved from 0.70000 to 0.87895, saving model to keras_model.h5


48/48 ━━━━━━━━━━━━━━━━━━━━ 27s 560ms/step - accuracy: 0.9159 - loss: 1.1007 - val_accuracy: 0.8789 - val_loss: 1.9067
Epoch 14/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 528ms/step - accuracy: 0.9321 - loss: 0.3517
Epoch 14: val_accuracy did not improve from 0.87895
48/48 ━━━━━━━━━━━━━━━━━━━━ 28s 576ms/step - accuracy: 0.9290 - loss: 0.3261 - val_accuracy: 0.8158 - val_loss: 3.2663
Epoch 15/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - accuracy: 0.9278 - loss: 0.2932
Epoch 15: val_accuracy improved from 0.87895 to 0.90526, saving model to keras_model.h5


48/48 ━━━━━━━━━━━━━━━━━━━━ 27s 549ms/step - accuracy: 0.9317 - loss: 0.2585 - val_accuracy: 0.9053 - val_loss: 0.4926
Training complete. Best model saved to keras_model.h5
